In [36]:
import hashlib
from Crypto import Random
from Crypto.Cipher import AES
from base64 import b64encode, b64decode
import binascii
import numpy as np
import pandas as pd
import time

In [37]:
class AESCipher(object):
    def __init__(self, key):
        self.block_size = AES.block_size
        self.key = hashlib.sha256(key.encode()).digest()

    def encrypt(self, plain_text):
        plain_text = self.__pad(plain_text)
        iv = Random.new().read(self.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        encrypted_text = cipher.encrypt(plain_text.encode())
        return b64encode(iv + encrypted_text).decode("utf-8")

    def decrypt(self, encrypted_text):
        encrypted_text = b64decode(encrypted_text)
        iv = encrypted_text[:self.block_size]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        plain_text = cipher.decrypt(encrypted_text[self.block_size:]).decode("utf-8")
        return self.__unpad(plain_text)

    def __pad(self, plain_text):
        number_of_bytes_to_pad = self.block_size - len(plain_text) % self.block_size
        ascii_string = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_string
        padded_plain_text = plain_text + padding_str
        return padded_plain_text

    @staticmethod
    def __unpad(plain_text):
        last_character = plain_text[len(plain_text) - 1:]
        return plain_text[:-ord(last_character)]

In [38]:
new=AESCipher("#Password1")

In [39]:
data="pviot key 1node dawnknkegkjwjqwJ JC VJVJKknwfklm ta"
enc=new.encrypt(data)
print(enc)

m9o9ebUIia1/kpikJWK5sH1P1q8C5ivi+EAtI7fDKFhMmbamZoIDssncKQLG7ZiZeG2M696WUKb5jNwBjIJr7m8AZe1gjvhBC3Hq4/z3Sso=


In [40]:
dec=new.decrypt(enc)
print(dec)

pviot key 1node dawnknkegkjwjqwJ JC VJVJKknwfklm ta


In [41]:
df=pd.read_csv("churn1.csv")

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RowNumber              10000 non-null  float64
 1   CustomerId             10000 non-null  object 
 2   CustomerId_ref_number  10000 non-null  float64
 3   Surname                10000 non-null  object 
 4   CreditScore            10000 non-null  float64
 5   Geography              10000 non-null  object 
 6   CountryCode            10000 non-null  object 
 7   Gender                 10000 non-null  object 
 8   Age                    10000 non-null  float64
 9   Tenure                 10000 non-null  float64
 10  Balance                10000 non-null  float64
 11  NumOfProducts          10000 non-null  float64
 12  HasCrCard              10000 non-null  float64
 13  IsActiveMember         10000 non-null  float64
 14  EstimatedSalary        10000 non-null  float64
 15

In [43]:
df.drop(columns=['RowNumber','CustomerId_ref_number','Unnamed: 16','Unnamed: 17','Unnamed: 18'],axis=1,inplace=True)

In [44]:
df.drop(df.index[10000:],inplace=True)

In [45]:
val=df.CustomerId

In [46]:
ans_enc=[]
ans_dec=[]
ans_time_enc=[]
ans_time_dec=[]
for i in range(len(val)):
    start1 = time.perf_counter()
    encrypted=new.encrypt(val[i])
    stop1 = time.perf_counter()
    #print(stop1,start1)
    #print("Time for encryption of ",val[i],"is ",stop1 - start1)
    #time.sleep(0.001)
    start2 = time.perf_counter()
    decrypted=new.decrypt(encrypted)
    stop2= time.perf_counter()
    #print(start2,stop2)
    #print("Time for decryption of ",decrypted,"is ",stop2 - start2)
    #print("Before Encryption: ",val[i])
    #print("After Encryption : ",encrypted)
    #print("Before Decryption: ",encrypted)
    #print("After Decryption : ",decrypted)
    #print()
    ans_time_enc.append(stop1-start1)
    ans_time_dec.append(stop2-start2)
    ans_enc.append(encrypted)
    ans_dec.append(decrypted)

In [48]:
name_enc=[]
name_dec=[]
name_time_enc=[]
name_time_dec=[]
val=df.Surname
for i in range(len(val)):
    start1 = time.perf_counter()
    encrypted=new.encrypt(val[i])
    stop1 = time.perf_counter()
    #time.sleep(0.3)
    start2 = time.perf_counter()
    decrypted=new.decrypt(encrypted)
    stop2= time.perf_counter()
    name_time_enc.append(stop1-start1)
    name_time_dec.append(stop2-start2)
    name_enc.append(encrypted)
    name_dec.append(decrypted)
    

In [49]:
encryption_time_customerid_series=pd.Series(ans_time_enc)
decryption_time_customerid_series=pd.Series(ans_time_dec)

encryption_time_surname_series=pd.Series(name_time_enc)
decryption_time_surname_series=pd.Series(name_time_dec)

In [50]:
temp_time_arr_enc_custid=np.array(ans_time_enc)
temp_time_arr_enc_surname=np.array(name_time_enc)
temp_time_arr_dec_surname=np.array(ans_time_dec)
temp_time_arr_dec_custid=np.array(name_time_dec)
print("There are ",len(name_enc),"values of surnames and customer IDs in the dataset")
print("Total time for encryption of Customer IDs is    : ",np.sum(temp_time_arr_enc_custid),"seconds")
print("Average time for encryption of 1 Customer ID is : ",np.average(temp_time_arr_enc_custid),"seconds")
print()
print("Total time decryption of customer IDs is        : ",np.sum(temp_time_arr_dec_custid),"seconds")
print("Average time for decryption of 1 customer ID is : ",np.average(temp_time_arr_dec_custid),"seconds")
print()
print("Total time for encryption of Surnames is        : ",np.sum(temp_time_arr_enc_surname),"seconds")
print("Average time for encryption of 1 Surname is     : ",np.average(temp_time_arr_enc_custid),"seconds")
print()
print("Total time for decryption of Surnames is        : ",np.sum(temp_time_arr_dec_surname),"seconds")
print('Average time for decryption of 1 Surname is     : ',np.average(temp_time_arr_dec_surname),"seconds")

There are  10000 values of surnames and customer IDs in the dataset
Total time for encryption of Customer IDs is    :  0.6111812999970425 seconds
Average time for encryption of 1 Customer ID is :  6.111812999970425e-05 seconds

Total time decryption of customer IDs is        :  0.31801159999486117 seconds
Average time for decryption of 1 customer ID is :  3.1801159999486116e-05 seconds

Total time for encryption of Surnames is        :  0.5839773000152491 seconds
Average time for encryption of 1 Surname is     :  6.111812999970425e-05 seconds

Total time for decryption of Surnames is        :  0.3100345000179914 seconds
Average time for decryption of 1 Surname is     :  3.100345000179914e-05 seconds
